In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16, ResNet50, InceptionResNetV2
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
random
import zipfile
import os

Using TensorFlow backend.


> # **1. Pre-processing Image** 

import file from /tmp

In [0]:
# #un-zip the dataset file
# local_zip = '/tmp/dataset.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/tmp')
# zip_ref.close()

# #return the shuffled image path 
# image_path = list(paths.list_images('/tmp/dataset/'))
# random.shuffle(image_path)

# #extract the lable, because the directory format is dataset/flower_name/image_name 
# # so the name of the flower will be extracted as follow:

# labels = [p.split(os.path.sep)[-2] for p in image_path]

**Import file from Drive upload**



In [0]:
#Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# #un-zip the dataset file
# local_zip = '/content/drive/My Drive/CV References/dataset.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/drive/My Drive/CV References/')
# zip_ref.close()


In [0]:
#return the shuffled image path 
image_path = list(paths.list_images('/content/drive/My Drive/Machine Learning/References/dataset')) 
random.shuffle(image_path)

#extract the lable, because the directory format is dataset/flower_name/image_name 
# so the name of the flower will be extracted as follow:

labels = [p.split(os.path.sep)[-2] for p in image_path]


In [0]:
# Encode
le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
# from glob import glob
# import re

# path = glob('/content/drive/My Drive/Machine Learning/CV References/dataset/*.jpg')
# list_image=[]
# for img_path in path:
#   # img_path = re.findall(r"\w.jpg", img_path)
#   img = cv2.imread(str(img_path))
#   img=np.expand_dims(img,0)
#   list_image.append(img)

In [8]:
path

[]

In [0]:
# Load image and resize for VGG model (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

> # **2.Build model**

In [0]:
# Load model VGG16. soft-max not included
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add layers
# output of VGG16 (last layer but not soft-max layer)
x = baseModel.output
# Flatten 
x = Flatten(name='flatten')(x)
# FC
x = Dense(1024, activation='relu')(x)
# Output layer with softmax activation
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(17, activation='softmax')(x)


model = model = Model(inputs=baseModel.input, outputs=x)



In [0]:
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

> # **3. Data Augmentation**

In [0]:
#Data Augmentation
# training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# test data
aug_test= ImageDataGenerator(rescale=1./255)

> # **4. Callback, Checkpoint and Early stopping**

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("/content/drive/My Drive/Machine Learning/CV References/dataset/Checkpoint.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', # value being monitored for improvement
                          min_delta = 0, #Abs value and is the min change required before we stop
                          patience = 3, #Number of epochs we wait before stopping 
                          verbose = 1,
                          restore_best_weights = True) #keeps the best weigths once stopped
                          
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]


> # **5.Training with all frozen layers**

In [0]:
# # freeze VGG16 model
# for layer in baseModel.layers:
#     layer.trainable = False
  

In [16]:

# model.compile('adam', 'categorical_crossentropy', ['accuracy'])
# numOfEpoch = 10
# H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
#                         steps_per_epoch=len(X_train)//32,
#                         validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
#                         validation_steps=len(X_test)//32,
#                         epochs=numOfEpoch,
#                         callbacks=callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
33/33 [==============================] - 14s 419ms/step - loss: 3.1947 - acc: 0.1879 - val_loss: 1.7332 - val_acc: 0.5039

Epoch 00001: val_loss improved from inf to 1.73323, saving model to /content/drive/My Drive/Machine Learning/CV References/dataset/Checkpoint.h5
Epoch 2/10
33/33 [==============================] - 11s 345ms/step - loss: 1.5080 - acc: 0.5229 - val_loss: 0.9239 - val_acc: 0.7333

Epoch 00002: val_loss improved from 1.73323 to 0.92387, saving model to /content/drive/My Drive/Machine Learning/CV References/dataset/Checkpoint.h5
Epoch 3/10
33/33 [==============================] - 11s 332ms/step - loss: 0.9730 - acc: 0.6898 - val_loss: 0.6648 - val_acc: 0.7917

Epoch 00003: val_loss improved from 0.92387 to 0.66482, saving model to /content/drive/My Drive/Machine Learning/CV References/dataset/Checkpoint.h5
Epoch 4/10
33/33 [==============================] - 11s 340

> # **6. Training with unfrozen layers**

In [0]:
# unfreeze last layer of VGG16 model
for layer in baseModel.layers[220:]:
    layer.trainable = True

In [23]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 10
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=4), 
                        steps_per_epoch=len(X_train)//4,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=4)),
                        validation_steps=len(X_test)//4,
                        epochs=numOfEpoch,
                        callbacks=callbacks)

Epoch 1/10
271/271 [==============================] - 20s 75ms/step - loss: 2.8356 - acc: 0.0452 - val_loss: 2.8358 - val_acc: 0.0404

Epoch 00001: val_loss did not improve from 2.83347
Epoch 2/10
271/271 [==============================] - 18s 65ms/step - loss: 2.8339 - acc: 0.0581 - val_loss: 2.8375 - val_acc: 0.0404

Epoch 00002: val_loss did not improve from 2.83347
Epoch 3/10
271/271 [==============================] - 18s 67ms/step - loss: 2.8333 - acc: 0.0581 - val_loss: 2.8389 - val_acc: 0.0331

Epoch 00003: val_loss did not improve from 2.83347
Epoch 4/10
271/271 [==============================] - 18s 66ms/step - loss: 2.8334 - acc: 0.0590 - val_loss: 2.8405 - val_acc: 0.0331
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 2.83347

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 00004: early stopping


In [21]:
from keras.models import load_model
smodel = load_model('/content/drive/My Drive/Machine Learning/CV References/dataset/Checkpoint.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


